In [1]:
import pandas as pd
import numpy as np
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

from atcs import ATCS
import fig_plot as fp


In [2]:
N_sample = 20
seed = 1
solver_type = 'minlp'
model_type = 'stochastic'
sample_type = f'head_{N_sample}'
data = ATCS(seed = 1)
data.choose_subset_point(N_sample, randomized = True) # Choose subset data

In [3]:
data.cc_df.head()

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99
0,1,0,1,1,0,1,0,0,1,0,...,1,0,1,0,1,1,0,0,1,0
1,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,1
3,0,0,0,0,1,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0,0,1,0,0,0,1,1,1,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
data.r_s_sub_df

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99
index,,,,,,,,,,,,,,,,,,,,,
120,127.887414,136.581700,154.326413,115.480101,61.025653,37.491349,98.657490,15.192927,26.594398,18.477441,...,34.487621,124.956789,103.931608,73.703623,45.072489,59.568732,27.374723,127.143206,102.345678,74.599497
552,60.391929,70.099256,116.496025,122.958761,26.489952,150.290209,52.670185,82.928970,20.462132,52.967941,...,66.030852,77.486963,30.003196,75.068774,79.557390,68.710861,113.349637,114.235948,60.014253,87.661647
90,130.136422,24.476656,26.185891,103.920026,90.088482,99.452652,70.306534,104.400772,79.987977,168.096573,...,110.918308,46.750488,29.105932,90.847656,108.279547,98.378269,45.825757,15.206112,34.995325,104.492064
973,65.957089,76.688249,95.611179,79.356057,122.850279,67.589396,116.076906,106.136351,86.791025,81.755160,...,119.423505,55.238895,63.129237,79.158360,102.645140,44.072141,31.644909,24.522991,150.866600,92.796120
667,41.008548,112.833172,64.437598,99.450087,43.375160,62.159440,71.285766,139.735059,45.403809,86.451207,...,119.856087,52.954027,150.268821,111.755181,66.379840,32.860638,111.651998,33.854267,48.680542,13.419703
314,50.515308,69.593849,77.108193,133.120716,84.294661,87.012312,36.215246,93.217115,55.807890,60.824726,...,93.512456,90.382710,86.960584,139.220440,107.072056,105.266039,54.678948,62.111419,68.895401,50.425168
181,30.119770,65.415479,117.918316,105.494660,98.125721,24.234452,84.712331,101.057454,39.708438,98.533258,...,45.259495,133.497828,52.224541,43.344313,130.411821,59.502931,84.026786,60.118870,83.289230,98.881910
119,106.700303,128.392683,71.880450,72.031055,82.003471,83.570231,50.373764,76.945021,77.765535,81.492192,...,42.403665,114.042567,79.703696,75.031229,31.912320,163.090000,60.614263,65.188720,86.351043,79.454244
49,25.688149,26.425672,130.398309,80.681628,109.331374,66.532928,87.021663,84.816290,152.833779,45.784124,...,90.197484,51.684908,72.743837,79.328367,13.891013,93.669229,101.001620,27.214222,53.041713,69.505378


In [6]:
location_fig = fp.create_l_fig()
location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
location_fig.show()

In [ ]:
# Stochastic MINLP Model
# Data
robot_id = data.r_s_sub_df.index.to_numpy()
robot_loc = data.l_sub_df.to_numpy()
# robot_range = data.R_Kub_df.to_numpy().flatten()
x_min,x_max = np.min(robot_loc, axis=0)[0], np.max(robot_loc, axis=0)[0]
y_min,y_max = np.min(robot_loc, axis=0)[1], np.max(robot_loc, axis=0)[1]
# dist_matrix = data.get_distance_matrix(sample_subset = True)

# Creates an empty Model
model = xp.problem(name='MINLP_Stochastic')
# Given Inputs and Parameters
R_K = {(i, col): data.r_s_sub_df.at[i, col] for i in data.r_s_sub_df.index for col in data.r_s_sub_df.columns}
L_Range = data.r_min # km
U_Range = data.r_max # km
U_Robot = data.q # Max robots per charger
U_Charger = data.m # Max chargers per station
L_Station = int(np.ceil(N_sample/(U_Charger * U_Robot))) # Maximum Robots per opened station
Dist_Max = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)
C_b = data.c_b # Investment cost per station
C_h = data.c_h # Cost of moving a robot
C_m = data.c_m # Maintenance cost per charger
C_c = data.c_c # Charging cost per km

Long_V, Lat_V = data.l_sub_df['longitude'].to_dict(), data.l_sub_df['latitude'].to_dict()
A_dict = {(i, col): data.cc_df.at[i, col] for i in data.cc_df.index for col in data.cc_df.columns}

# Parameters
# U_Station = L_Station + 1 # For now
U_Station = len(robot_id)
data.set_output_folder(solver_type, model_type, name = f'{N_sample}_{sample_type}samp_{U_Station}s_seed{seed}')

# Sets and Notation
V = data.r_s_sub_df.index.to_numpy() # Robot Vehicle sets
S = np.arange(U_Station)
K = data.r_s_sub_df.columns.to_list()


# Output Variables
x = model.addVariables(S, name = 'x', vartype = xp.continuous, lb = x_min, ub = x_max) # Longitude of station s
y = model.addVariables(S, name = 'y', vartype = xp.continuous, lb = y_min, ub = y_max) # Latitude of station s

eta = model.addVariables(S, name = 'eta', vartype = xp.integer) # Number of charger installed at station s
mu = model.addVariables(S, name = 'mu', vartype = xp.binary) # Whether station s is built
beta = model.addVariables(V, S, K, name = 'beta', vartype = xp.binary) # Whether robot v is covered by station s
alpha = model.addVariables(V, S, K, name = 'alpha', vartype = xp.binary) # Whether robot v is brought to station s by taking penalty
d = model.addVariables(V, S, K, name = 'd', vartype = xp.continuous, lb = 0) # Distance from v to s


# Robot v can be allocated to charging station s iff it is built
model.addConstraint(beta[v,s,k] <= A_dict[v,k] * mu[s] for v in V for s in S for k in K) 

model.addConstraint(xp.Sum(mu[s] for s in S) >= L_Station)
model.addConstraint(xp.Sum(mu[s] for s in S) <= L_Station + 2)
# Each Robot v must be allocated to only one station
model.addConstraint(xp.Sum(beta[v,s,k] for s in S) == 1 * A_dict[v,k] for v in V for k in K)

# Number of chargers per station
model.addConstraint(eta[s] >= xp.Sum(beta[v,s,k] for v in V)/U_Robot for s in S for k in K)
model.addConstraint(eta[s] <= U_Charger for s in S)

# Robot v can be brought to station s by human iff the robot is allocated to that station
model.addConstraint(alpha[v,s,k] <= beta[v,s,k] for v in V for s in S for k in K)

model.addConstraint(d[v,s,k] <= R_K[v,k] * (1 - alpha[v,s,k]) for v in V for s in S for k in K)
model.addConstraint(d[v,s,k] >= xp.sqrt((Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2)
                                - Dist_Max * (alpha[v,s,k] + (1 - beta[v,s,k])) 
                                        for v in V for s in S for k in K)
model.addConstraint(d[v,s,k] <= Dist_Max * beta[v,s,k] for v in V for s in S for k in K)

# Objective Function
Build_station_cost = xp.Sum(C_b * mu[s] for s in S)
Build_charger_cost = xp.Sum(C_m * eta[s] for s in S)



In [8]:
data.r_s_sub_df

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99
index,,,,,,,,,,,,,,,,,,,,,
120,127.887414,136.581700,154.326413,115.480101,61.025653,37.491349,98.657490,15.192927,26.594398,18.477441,...,34.487621,124.956789,103.931608,73.703623,45.072489,59.568732,27.374723,127.143206,102.345678,74.599497
552,60.391929,70.099256,116.496025,122.958761,26.489952,150.290209,52.670185,82.928970,20.462132,52.967941,...,66.030852,77.486963,30.003196,75.068774,79.557390,68.710861,113.349637,114.235948,60.014253,87.661647
90,130.136422,24.476656,26.185891,103.920026,90.088482,99.452652,70.306534,104.400772,79.987977,168.096573,...,110.918308,46.750488,29.105932,90.847656,108.279547,98.378269,45.825757,15.206112,34.995325,104.492064
973,65.957089,76.688249,95.611179,79.356057,122.850279,67.589396,116.076906,106.136351,86.791025,81.755160,...,119.423505,55.238895,63.129237,79.158360,102.645140,44.072141,31.644909,24.522991,150.866600,92.796120
667,41.008548,112.833172,64.437598,99.450087,43.375160,62.159440,71.285766,139.735059,45.403809,86.451207,...,119.856087,52.954027,150.268821,111.755181,66.379840,32.860638,111.651998,33.854267,48.680542,13.419703
314,50.515308,69.593849,77.108193,133.120716,84.294661,87.012312,36.215246,93.217115,55.807890,60.824726,...,93.512456,90.382710,86.960584,139.220440,107.072056,105.266039,54.678948,62.111419,68.895401,50.425168
181,30.119770,65.415479,117.918316,105.494660,98.125721,24.234452,84.712331,101.057454,39.708438,98.533258,...,45.259495,133.497828,52.224541,43.344313,130.411821,59.502931,84.026786,60.118870,83.289230,98.881910
119,106.700303,128.392683,71.880450,72.031055,82.003471,83.570231,50.373764,76.945021,77.765535,81.492192,...,42.403665,114.042567,79.703696,75.031229,31.912320,163.090000,60.614263,65.188720,86.351043,79.454244
49,25.688149,26.425672,130.398309,80.681628,109.331374,66.532928,87.021663,84.816290,152.833779,45.784124,...,90.197484,51.684908,72.743837,79.328367,13.891013,93.669229,101.001620,27.214222,53.041713,69.505378


In [9]:
R_K

{(120, 's0'): 127.88741372882644,
 (120, 's1'): 136.58170016923816,
 (120, 's2'): 154.32641257645275,
 (120, 's3'): 115.48010059640907,
 (120, 's4'): 61.02565341487305,
 (120, 's5'): 37.491348770258845,
 (120, 's6'): 98.6574902303278,
 (120, 's7'): 15.19292683324013,
 (120, 's8'): 26.59439752534084,
 (120, 's9'): 18.477440612634048,
 (120, 's10'): 37.80180801766148,
 (120, 's11'): 102.86552541993228,
 (120, 's12'): 172.333653959637,
 (120, 's13'): 50.64445603712759,
 (120, 's14'): 67.34890806949026,
 (120, 's15'): 53.65606739626996,
 (120, 's16'): 65.8347163759948,
 (120, 's17'): 36.91552865350003,
 (120, 's18'): 35.873394060107145,
 (120, 's19'): 69.74115781915005,
 (120, 's20'): 52.13376869522368,
 (120, 's21'): 47.43415885080681,
 (120, 's22'): 89.35237554495662,
 (120, 's23'): 111.59103444667538,
 (120, 's24'): 54.341794934714486,
 (120, 's25'): 21.86804174853713,
 (120, 's26'): 99.61572478346856,
 (120, 's27'): 37.472810666822376,
 (120, 's28'): 64.2488107619846,
 (120, 's29'): 10

In [10]:
data.r_s_sub_df

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99
index,,,,,,,,,,,,,,,,,,,,,
120,127.887414,136.581700,154.326413,115.480101,61.025653,37.491349,98.657490,15.192927,26.594398,18.477441,...,34.487621,124.956789,103.931608,73.703623,45.072489,59.568732,27.374723,127.143206,102.345678,74.599497
552,60.391929,70.099256,116.496025,122.958761,26.489952,150.290209,52.670185,82.928970,20.462132,52.967941,...,66.030852,77.486963,30.003196,75.068774,79.557390,68.710861,113.349637,114.235948,60.014253,87.661647
90,130.136422,24.476656,26.185891,103.920026,90.088482,99.452652,70.306534,104.400772,79.987977,168.096573,...,110.918308,46.750488,29.105932,90.847656,108.279547,98.378269,45.825757,15.206112,34.995325,104.492064
973,65.957089,76.688249,95.611179,79.356057,122.850279,67.589396,116.076906,106.136351,86.791025,81.755160,...,119.423505,55.238895,63.129237,79.158360,102.645140,44.072141,31.644909,24.522991,150.866600,92.796120
667,41.008548,112.833172,64.437598,99.450087,43.375160,62.159440,71.285766,139.735059,45.403809,86.451207,...,119.856087,52.954027,150.268821,111.755181,66.379840,32.860638,111.651998,33.854267,48.680542,13.419703
314,50.515308,69.593849,77.108193,133.120716,84.294661,87.012312,36.215246,93.217115,55.807890,60.824726,...,93.512456,90.382710,86.960584,139.220440,107.072056,105.266039,54.678948,62.111419,68.895401,50.425168
181,30.119770,65.415479,117.918316,105.494660,98.125721,24.234452,84.712331,101.057454,39.708438,98.533258,...,45.259495,133.497828,52.224541,43.344313,130.411821,59.502931,84.026786,60.118870,83.289230,98.881910
119,106.700303,128.392683,71.880450,72.031055,82.003471,83.570231,50.373764,76.945021,77.765535,81.492192,...,42.403665,114.042567,79.703696,75.031229,31.912320,163.090000,60.614263,65.188720,86.351043,79.454244
49,25.688149,26.425672,130.398309,80.681628,109.331374,66.532928,87.021663,84.816290,152.833779,45.784124,...,90.197484,51.684908,72.743837,79.328367,13.891013,93.669229,101.001620,27.214222,53.041713,69.505378
